In [1]:
import os
import pandas as pd
import scipy.stats as sci
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.multitest import multipletests
import warnings
import glob
import sys
curr_path = os.getcwd()
sys.path.append(f"{curr_path}")

# load structural data

# structural_results_path = "/Users/hkassandra/Desktop/WAPIAW/Results/meta_analysis_with_covars.csv"

# df_structural = pd.read_csv(structural_results_path)
# df_structural['estimate'] = abs(df_structural['estimate'])

# df_volume = df_structural[(df_structural["IDP_name"].str.contains("volume")) | (df_structural["Yeo_name"] == "Subcortical")]
# df_area = df_structural[(df_structural["IDP_name"].str.contains("area")) & (df_structural["Yeo_name"] != "Subcortical")]
# df_thickness = df_structural[(df_structural["IDP_name"].str.contains("thickness")) & (df_structural["Yeo_name"] != "Subcortical")]

df_volume = pd.read_csv("Results/meta_results_volume/meta_analysis_without_covars.csv")
df_area = pd.read_csv("Results/meta_results_area/meta_analysis_without_covars.csv")
df_thickness = pd.read_csv("Results/meta_results_thickness/meta_analysis_without_covars.csv")

structural_dfs = [df_volume, df_area, df_thickness]

# load functional data

schaefer_partial_path = "Results/schaefer_partials/meta_analysis_without_covars_with_yeo.csv"
schaefer_partial_df = pd.read_csv(schaefer_partial_path)
schaefer_full_path = "Results/schaefer_full/meta_analysis_without_covars_with_yeo.csv"
schaefer_full_df = pd.read_csv(schaefer_full_path) 

functional_dfs = [schaefer_partial_df, schaefer_full_df]

df_volume["estimate"] = df_volume["estimate"].abs()
df_volume["imaging_metric"] = "Gray Matter Volume"
df_area["estimate"] = df_area["estimate"].abs()
df_area["imaging_metric"] = "Cortical Surface Area" 
df_thickness["estimate"] = df_thickness["estimate"].abs()
df_thickness["imaging_metric"] = "Cortical Thickness"
schaefer_partial_df["estimate"] = schaefer_partial_df["estimate"].abs()
schaefer_partial_df["imaging_metric"] = "Partial Netmats (Schaefer)"
schaefer_full_df["estimate"] = schaefer_full_df["estimate"].abs()
schaefer_full_df["imaging_metric"] = "Full Netmats (Schaefer)"

yeo_mapping = {
    1: "Visual",
    2: "Somatomotor",
    3: "Dorsal Attention",
    4: "Ventral Attention",
    5: "Limbic",
    6: "Frontoparietal",
    7: "Default",
    8: "Subcortical"
}
schaefer_partial_df['Yeo'] = schaefer_partial_df['Yeo1']
schaefer_partial_df['Yeo_name'] = schaefer_partial_df['Yeo1'].map(yeo_mapping)
schaefer_full_df['Yeo'] = schaefer_full_df['Yeo1']
schaefer_full_df['Yeo_name'] = schaefer_full_df['Yeo1'].map(yeo_mapping)

df_total = pd.concat([df_volume, df_area, df_thickness, schaefer_partial_df, schaefer_full_df], ignore_index=True)

print(df_total.head())

print(schaefer_partial_df.columns)
print(schaefer_partial_df["phen_group"].unique())
print(max(schaefer_partial_df["estimate"]))

all_dfs = structural_dfs + functional_dfs

        name  estimate        se   z-score   p-value  ci_0.025  ci_0.975  \
0  intercept  0.008897  0.007243 -1.228264  0.219348 -0.023093  0.005300   
1  intercept  0.019102  0.008982 -2.126764  0.033440 -0.036707 -0.001498   
2  intercept  0.024409  0.011987 -2.036239  0.041726 -0.047904 -0.000914   
3  intercept  0.012874  0.016129 -0.798223  0.424741 -0.044486  0.018737   
4  intercept  0.017651  0.007091 -2.489250  0.012801 -0.031549 -0.003753   

                   IDP_name   phen_group  Yeo Yeo_name     p_fdr  \
0           rhlingualvolume  Neuroticism  1.0   Visual  0.322814   
1   rhparahippocampalvolume  Neuroticism  1.0   Visual  0.072453   
2          rhfusiformvolume  Neuroticism  1.0   Visual  0.083453   
3  rhlateraloccipitalvolume  Neuroticism  1.0   Visual  0.534352   
4     rhpericalcarinevolume  Neuroticism  1.0   Visual  0.034431   

   Significance      imaging_metric  Unnamed: 0  Node1  Node2  Yeo1  Yeo2  
0         False  Gray Matter Volume         NaN    NaN    

In [2]:
### PHENO --> 6X main effect of depression phenotype (with FDR correction) 

# List to store p-values and imaging metrics from ANOVA tests
all_img_metrics = []
p_values = []

# First loop: Collect p-values
for df in all_dfs:
    model = ols("""estimate ~ C(phen_group)""", data=df).fit()
    anova_results = sm.stats.anova_lm(model, typ=2)
    img_met = df["imaging_metric"].unique()[0]
    all_img_metrics.append(img_met)

    # Extract the p-value from the ANOVA results
    p_value = anova_results['PR(>F)'][0]
    p_values.append(p_value)

    # Post hoc for Yeo_name
    mc = MultiComparison(df['estimate'], df['phen_group'])
    posthoc_results = mc.tukeyhsd()

# Apply FDR correction to the collected p-values
_, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

# Second loop: Append FDR-corrected p-values to the ANOVA results and print
corrected_p_index = 0
for df in all_dfs:
    model = ols("""estimate ~ C(phen_group)""", data=df).fit()
    anova_results = sm.stats.anova_lm(model, typ=2)
    img_met = df["imaging_metric"].unique()[0]

    # Add a new column for FDR corrected p-values
    anova_results['P_FDR'] = [corrected_p_values[corrected_p_index]] + [None] * (len(anova_results) - 1)
    corrected_p_index += 1

    # Print the anova_results with the new column
    print(f"\n\n One-Way ANOVA with depression phenotype group on {img_met} results - with FDR correction \n\n")
    print(anova_results)

    # Post hoc for phen_group
    mc = MultiComparison(df['estimate'], df['phen_group'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for phen_group - absolute value \n\n")
    print(posthoc_results)

/var/folders/tx/3z8_jq352fd7z_0llwbf29380000gp/T/ipykernel_650/1067996279.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_results['PR(>F)'][0]
/var/folders/tx/3z8_jq352fd7z_0llwbf29380000gp/T/ipykernel_650/1067996279.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_results['PR(>F)'][0]
/var/folders/tx/3z8_jq352fd7z_0llwbf29380000gp/T/ipykernel_650/1067996279.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use



 One-Way ANOVA with depression phenotype group on Gray Matter Volume results - with FDR correction 


                 sum_sq     df         F    PR(>F)     P_FDR
C(phen_group)  0.000609    1.0  3.520366  0.062511  0.104186
Residual       0.026630  154.0       NaN       NaN       NaN


 Post Hoc for phen_group - absolute value 


    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
  group1      group2   meandiff p-adj   lower  upper  reject
------------------------------------------------------------
Depression Neuroticism   -0.004 0.0625 -0.0081 0.0002  False
------------------------------------------------------------


 One-Way ANOVA with depression phenotype group on Cortical Surface Area results - with FDR correction 


                 sum_sq     df        F    PR(>F)     P_FDR
C(phen_group)  0.000298    1.0  1.79844  0.182393  0.227991
Residual       0.020187  122.0      NaN       NaN       NaN


 Post Hoc for phen_group - absolute value 


    Multiple Comparison of 

In [3]:
# perform two-way ANOVA
model = ols("""estimate ~ C(phen_group) + C(imaging_metric) + C(phen_group):C(imaging_metric)""", data=df_total).fit()
print("\n\n Two-Way ANOVA with phen_group and image data type (absolute value) \n\n")
print((sm.stats.anova_lm(model, typ=2)))

# post_hoc for phen_group
mc = MultiComparison(df_total['estimate'], df_total['phen_group'])
posthoc_results = mc.tukeyhsd()
print("\n\n Post Hoc for phen_group - absolute value \n\n")
print(posthoc_results)

# post_hoc for brain data type
mc = MultiComparison(df_total['estimate'], df_total['imaging_metric'])
posthoc_results = mc.tukeyhsd()
print("\n\n Post Hoc for imaging metric (absolute val) \n\n")
print(posthoc_results)

# post_hoc for brain data type
mc = MultiComparison(df_total['estimate'], df_total['imaging_metric']+df_total['phen_group'])
posthoc_results = mc.tukeyhsd()
print("\n\n Post Hoc for interaction effect (absolute val) \n\n")
print(posthoc_results)



 Two-Way ANOVA with phen_group and image data type (absolute value) 


                                   sum_sq        df            F        PR(>F)
C(phen_group)                    0.007131       1.0   198.088055  5.767500e-45
C(imaging_metric)                0.731713       4.0  5081.666750  0.000000e+00
C(phen_group):C(imaging_metric)  0.013232       4.0    91.896191  3.367201e-78
Residual                         6.472166  179794.0          NaN           NaN


 Post Hoc for phen_group - absolute value 


   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1      group2   meandiff p-adj lower  upper  reject
----------------------------------------------------------
Depression Neuroticism   0.0004   0.0 0.0003 0.0005   True
----------------------------------------------------------


 Post Hoc for imaging metric (absolute val) 


                   Multiple Comparison of Means - Tukey HSD, FWER=0.05                   
         group1                   group2           

In [4]:
### SPATIAL --> 6X main effect of yeo and depression phenotype (with FDR correction) 

# List to store p-values from ANOVA tests
all_p_values = []

for df in all_dfs:
    # Perform two-way ANOVA
    model = ols("""estimate ~ C(phen_group) + C(Yeo_name) + C(phen_group):C(Yeo_name)""", data=df).fit()
    img_met = df["imaging_metric"].unique()[0]
    anova_results = sm.stats.anova_lm(model, typ=2)
    print(f"\n\n Two-Way ANOVA with Yeo_name and phen_group on {img_met} results - absolute value \n\n")
    print(anova_results)

    # Collect p-values for FDR correction
    p_values = anova_results['PR(>F)'].tolist()
    all_p_values.extend(p_values[:-1])  # Exclude the last p-value which is NaN for the residual

    # Post hoc for Yeo_name
    mc = MultiComparison(df['estimate'], df['Yeo_name'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for Yeo_name - absolute value \n\n")
    print(posthoc_results)

    # Post hoc for phen_group
    mc = MultiComparison(df['estimate'], df['phen_group'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for phen_group - absolute value \n\n")
    print(posthoc_results)

    # post hoc for interaction effect
    mc = MultiComparison(df['estimate'], df['Yeo_name']+df['phen_group'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for phen_group - absolute value \n\n")
    print(posthoc_results)

# # Apply FDR correction to the collected p-values
# _, corrected_p_values, _, _ = multipletests(all_p_values, alpha=0.05, method='fdr_bh')

# # Iterate through the dataframes again to add the corrected p-values
# corrected_p_index = 0
# for df in all_dfs:
#     # Perform two-way ANOVA again to get the anova_results DataFrame
#     model = ols("""estimate ~ C(phen_group) + C(Yeo_name) + C(phen_group):C(Yeo_name)""", data=df).fit()
#     anova_results = sm.stats.anova_lm(model, typ=2)
#     img_met = df["imaging_metric"].unique()[0]

#     # Add a new column for FDR corrected p-values
#     num_effects = len(anova_results) - 1  # Exclude the residual row
#     anova_results['P_FDR'] = [None] * len(anova_results)  # Initialize with None
#     anova_results.iloc[:num_effects, anova_results.columns.get_loc('P_FDR')] = corrected_p_values[corrected_p_index:corrected_p_index + num_effects]
#     corrected_p_index += num_effects

#     # Print the anova_results with the new column
#     print(f"\n\n Two-Way ANOVA with Yeo_name and phen_group on {img_met} results - with FDR correction \n\n")
#     print(anova_results)

#         # Post hoc for Yeo_name
#     mc = MultiComparison(df['estimate'], df['Yeo_name'])
#     posthoc_results = mc.tukeyhsd()
#     print("\n\n Post Hoc for Yeo_name - absolute value \n\n")
#     print(posthoc_results)

#     # Post hoc for phen_group
#     mc = MultiComparison(df['estimate'], df['phen_group'])
#     posthoc_results = mc.tukeyhsd()
#     print("\n\n Post Hoc for phen_group - absolute value \n\n")
#     print(posthoc_results)

#     # post hoc for interaction effect
#     mc = MultiComparison(df['estimate'], df['Yeo_name']+df['phen_group'])
#     posthoc_results = mc.tukeyhsd()
#     print("\n\n Post Hoc for phen_group - absolute value \n\n")
#     print(posthoc_results)



 Two-Way ANOVA with Yeo_name and phen_group on Gray Matter Volume results - absolute value 


                             sum_sq     df          F        PR(>F)
C(phen_group)              0.000609    1.0   5.164601  2.457491e-02
C(Yeo_name)                0.008324    7.0  10.088409  3.516689e-10
C(phen_group):C(Yeo_name)  0.001804    7.0   2.186996  3.886734e-02
Residual                   0.016502  140.0        NaN           NaN


 Post Hoc for Yeo_name - absolute value 


           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2      meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------
          Default  Dorsal Attention  -0.0108 0.6096 -0.0291  0.0075  False
          Default    Frontoparietal   0.0038 0.9898 -0.0098  0.0173  False
          Default            Limbic  -0.0047 0.8642 -0.0151  0.0058  False
          Default       Somatomotor  -0.0036   0.95 -0.0132  0.0061  

In [5]:
### SPATIAL --> 6X main effect of yeo only (with FDR correction) 

# List to store p-values and imaging metrics from ANOVA tests
all_img_metrics = []
p_values = []

# First loop: Collect p-values
for df in all_dfs:
    model = ols("""estimate ~ C(Yeo_name)""", data=df).fit()
    anova_results = sm.stats.anova_lm(model, typ=2)
    img_met = df["imaging_metric"].unique()[0]
    all_img_metrics.append(img_met)

    # Extract the p-value from the ANOVA results
    p_value = anova_results['PR(>F)'][0]
    p_values.append(p_value)

    # Post hoc for Yeo_name
    mc = MultiComparison(df['estimate'], df['Yeo_name'])
    posthoc_results = mc.tukeyhsd()

# Apply FDR correction to the collected p-values
_, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

# Second loop: Append FDR-corrected p-values to the ANOVA results and print
corrected_p_index = 0
for df in all_dfs:
    model = ols("""estimate ~ C(Yeo_name)""", data=df).fit()
    anova_results = sm.stats.anova_lm(model, typ=2)
    img_met = df["imaging_metric"].unique()[0]

    # Add a new column for FDR corrected p-values
    anova_results['P_FDR'] = [corrected_p_values[corrected_p_index]] + [None] * (len(anova_results) - 1)
    corrected_p_index += 1

    # Print the anova_results with the new column
    print(f"\n\n One-Way ANOVA with Yeo network on {img_met} results - with FDR correction \n\n")
    print(anova_results)

    # Post hoc for phen_group
    mc = MultiComparison(df['estimate'], df['Yeo_name'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for Yeo_name - absolute value \n\n")
    print(posthoc_results)

/var/folders/tx/3z8_jq352fd7z_0llwbf29380000gp/T/ipykernel_650/3159170281.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_results['PR(>F)'][0]
/var/folders/tx/3z8_jq352fd7z_0llwbf29380000gp/T/ipykernel_650/3159170281.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_results['PR(>F)'][0]
/var/folders/tx/3z8_jq352fd7z_0llwbf29380000gp/T/ipykernel_650/3159170281.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use



 One-Way ANOVA with Yeo network on Gray Matter Volume results - with FDR correction 


               sum_sq     df        F        PR(>F)         P_FDR
C(Yeo_name)  0.008324    7.0  9.30424  1.594013e-09  3.985032e-09
Residual     0.018915  148.0      NaN           NaN           NaN


 Post Hoc for Yeo_name - absolute value 


           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2      meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------
          Default  Dorsal Attention  -0.0108 0.6096 -0.0291  0.0075  False
          Default    Frontoparietal   0.0038 0.9898 -0.0098  0.0173  False
          Default            Limbic  -0.0047 0.8642 -0.0151  0.0058  False
          Default       Somatomotor  -0.0036   0.95 -0.0132  0.0061  False
          Default       Subcortical  -0.0192    0.0 -0.0277 -0.0108   True
          Default Ventral Attention  -0.0013 0.9999 -0.0113  0.0088 

In [ ]:
### old anovas code


for df in structural_dfs:
    abs_df = df.copy()
    abs_df["estimate"] = abs_df["estimate"].abs()

    #perform three-way ANOVA
    model = ols("""estimate ~ C(phen_group) + C(Yeo_name) + C(phen_group):C(Yeo_name)""", data=abs_df).fit()

    print("\n\n Two-Way ANOVA with Yeo_name and phen_group - absolute value \n\n")
    print((sm.stats.anova_lm(model, typ=2)))

    # post hoc for Yeo_name 
    mc = MultiComparison(abs_df['estimate'], abs_df['Yeo_name'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for Yeo_name - absolute value \n\n")
    print(posthoc_results)

    # post_hoc for phen_group
    mc = MultiComparison(abs_df['estimate'], abs_df['phen_group'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for phen_group - absolute value \n\n")
    print(posthoc_results)
    #two way ANOVA with main effect for network and main effect for phenotype group


print("begin functional anovas")
for df in functional_dfs:
    abs_df = df.copy()
    abs_df["estimate"] = abs_df["estimate"].abs()

    #perform three-way ANOVA
    model = ols("""estimate ~ C(phen_group) + C(Yeo_name) + C(phen_group):C(Yeo_name)""", data=abs_df).fit()

    print("\n\n Two-Way ANOVA with Yeo_name and phen_group - absolute value \n\n")
    print((sm.stats.anova_lm(model, typ=2)))

    # post hoc for Yeo_name 
    mc = MultiComparison(abs_df['estimate'], abs_df['Yeo_name'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for Yeo_name - absolute value \n\n")
    print(posthoc_results)

    # post_hoc for phen_group
    mc = MultiComparison(abs_df['estimate'], abs_df['phen_group'])
    posthoc_results = mc.tukeyhsd()
    print("\n\n Post Hoc for phen_group - absolute value \n\n")
    print(posthoc_results)
    #two way ANOVA with main effect for network and main effect for phenotype group





In [ ]:
### old anovas code


print("ANOVAs on combined meta-analysis results across all imaging metrics")

df_volume["estimate"] = df_volume["estimate"].abs()
df_volume["imaging_metric"] = "volume"
df_area["estimate"] = df_area["estimate"].abs()
df_area["imaging_metric"] = "area" 
df_thickness["estimate"] = df_thickness["estimate"].abs()
df_thickness["imaging_metric"] = "thickness"
amp_df["estimate"] = amp_df["estimate"].abs()
amp_df["imaging_metric"] = "amplitude"
fc_df["estimate"] = fc_df["estimate"].abs()
fc_df["imaging_metric"] = "partial netmats"

df_total = pd.concat([df_volume, df_area, df_thickness, amp_df, fc_df], ignore_index=True)
# print(df_total.columns)

# perform three-way ANOVA
model = ols("""estimate ~ C(phen_group) + C(Yeo_name) + C(imaging_metric) +
            C(phen_group):C(Yeo_name) + C(phen_group):C(imaging_metric) + C(Yeo_name):C(imaging_metric)""", data=df_total).fit()
print("\n\n Three-Way ANOVA with Yeo_name phen_group and image data type (absolute value) \n\n")
print((sm.stats.anova_lm(model, typ=2)))

# post hoc for Yeo_name 
mc = MultiComparison(df_total['estimate'], df_total['Yeo_name'])
posthoc_results = mc.tukeyhsd()
print("\n\n Post Hoc for Yeo_name - absolute value \n\n")
print(posthoc_results)

# post_hoc for phen_group
mc = MultiComparison(df_total['estimate'], df_total['phen_group'])
posthoc_results = mc.tukeyhsd()
print("\n\n Post Hoc for phen_group - absolute value \n\n")
print(posthoc_results)

# post_hoc for brain data type
mc = MultiComparison(df_total['estimate'], df_total['imaging_metric'])
posthoc_results = mc.tukeyhsd()
print("\n\n Post Hoc for imaging metric (absolute val) \n\n")
print(posthoc_results)